In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import paderbox as pb
from paderbox.array import interval
from padercontrib.database.fearless import Fearless
from padertorch import Model
from paderbox.transform import stft,fbank
import scipy

In [169]:
import torch
import torch.nn as nn
from torch import Tensor
from collections import OrderedDict 

In [170]:
import itertools
import padertorch as pt
import padercontrib as pc
import paderbox as pb

from pathlib import Path
from padertorch import Model
from tqdm import tqdm
from pprint import pprint

In [171]:
path = '/net/vol/saadmann/dump_audio_test/FS02_dev_001.wav'
rre_audio = np.round(pb.io.load_audio(path)).astype(bool)
rre_audio

array([False, False, False, ..., False, False, False])

In [172]:
pb.io.load_audio('/net/vol/saadmann/dump_audio_test/FS02_dev_001.wav')

array([0., 0., 0., ..., 0., 0., 0.])

In [173]:
intervals8k = interval.ArrayInterval(rre_audio)
intervals8k

ArrayInterval("1709760:1744880, 1952000:1969040, 2812480:2820720, 2827840:2994480, 2999040:3004400, 3016960:3021040, 3779840:3788400, 3794560:3846640, 4512000:4520560, 4532160:4590960, 4592800:4593520, 4768000:4803120, 6613120:6620240, 6623200:6624000, 6655200:6664240, 6664640:6726960, 6728800:6730480, 6734080:6790960, 6791200:6791920, 6793120:6900400, 6904320:6905200, 6905280:6946160, 7104000:7260880, 7635680:7644400, 7648000:7683120, 7744000:7744720, 7745280:7797840, 10959200:11012400, 11344160:11351760, 11371200:11400560, 12802560:12898320, 12985600:13065360, 13973440:13982480, 13983360:14019920", shape=(14816000,))

In [174]:
db = Fearless()
dataset_Dev_str = db.get_dataset('Dev_stream')
df_Dev_str = pd.DataFrame(dataset_Dev_str)

In [175]:
x_train_raw_audio = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                     start = intervals8k.intervals[0][0], 
                                                    stop = intervals8k.intervals[0][1]).astype(np.float32)
pb.io.play(x_train_raw_audio, sample_rate=8000)

In [121]:
dataset_Dev_str[0]['transcription'][0:4], dataset_Dev_str[0]['speaker_id'][0:4]

(['FLIGHT PROCEDURES.',
  'Go PROCEDURES.',
  'Upper clocks had C.S.M. SEP',
  'Rog.'],
 ['PROCEDURES1', 'FD1', 'PROCEDURES1', 'FD1'])

In [122]:
#To-do: pandas dataframe of Dev-set:
db = Fearless()
dataset_Dev_str = db.get_dataset('Dev_stream')
df_Dev_str = pd.DataFrame(dataset_Dev_str)
df_Dev_str.head()

,audio_path,end,num_samples,num_speakers,speaker_id,start,transcription,example_id,dataset
0,{'observation': '/net/db/fearless/Audio/Stream...,"[1716960, 1724240, 1738480, 1742720, 1967600, ...",14816000,8,"[PROCEDURES1, FD1, PROCEDURES1, FD1, CONTROL1,...","[1710480, 1716960, 1724720, 1739440, 1955360, ...","[FLIGHT PROCEDURES., Go PROCEDURES., Upper clo...",FS02_dev_001,Dev_stream
1,{'observation': '/net/db/fearless/Audio/Stream...,"[24800, 128560, 143040, 164080, 172320, 206240...",14400000,30,"[EECOM3, TRACK0, MADRID, TRACK0, EECOM3, MADRI...","[16800, 123360, 140320, 148560, 164080, 176000...","[SPAN EECOM conference., MADRID TRACK., MADRID...",FS02_dev_002,Dev_stream
2,{'observation': '/net/db/fearless/Audio/Stream...,"[98400, 158960, 169680, 262480, 526720, 767200...",14400000,26,"[CAPCOM1, EMU, FD1, BUZZ, BUZZ, BUZZ, NEIL, BU...","[36320, 98400, 162720, 169680, 285840, 544480,...","[Uh roger BUZZ, and break break COLUMBIA this ...",FS02_dev_003,Dev_stream
3,{'observation': '/net/db/fearless/Audio/Stream...,"[748080, 760480, 770000, 802720, 826080, 88120...",14955294,11,"[RETRO1, FIDO1, RETRO1, UNK, FIDO1, RETRO1, FI...","[739600, 757520, 761920, 797040, 815840, 84056...","[FIDO this is RETRO., Go ahead., Did you get t...",FS02_dev_004,Dev_stream
4,{'observation': '/net/db/fearless/Audio/Stream...,"[28240, 102000, 146320, 182160, 258720, 376480...",14400000,14,"[CAPCOM1, BUZZ, CAPCOM1, BUZZ, CAPCOM1, BUZZ, ...","[0, 61920, 103840, 172560, 190080, 337840, 390...",[Is HOUSTON uh radio check and verify T.V. cir...,FS02_dev_005,Dev_stream


In [206]:
print('speaker ID: ', df_Dev_str['speaker_id'][0][0], '\n',
      'transcription: ', df_Dev_str['transcription'][0][0], '\n',
      'start of sample: ', df_Dev_str['start'][0][0], '\n',
      'end of sample: ', df_Dev_str['end'][0][0], '\n',
      'number of samples: ', (df_Dev_str['end'][0][0]-df_Dev_str['start'][0][0]), '\n',
    'duration: ', (df_Dev_str['end'][0][0]-df_Dev_str['start'][0][0])/8000, 's')

speaker ID:  PROCEDURES1 
 transcription:  FLIGHT PROCEDURES. 
 start of sample:  1710480 
 end of sample:  1716960 
 number of samples:  6480 
 duration:  0.81 s


In [207]:
print('speaker ID: ', df_Dev_str['speaker_id'][0][1], '\n',
      'transcription: ', df_Dev_str['transcription'][0][1], '\n',
      'start of sample: ', df_Dev_str['start'][0][1], '\n',
      'end of sample: ', df_Dev_str['end'][0][1], '\n',
      'number of samples: ', (df_Dev_str['end'][0][1]-df_Dev_str['start'][0][1]), '\n',
    'duration: ', (df_Dev_str['end'][0][1]-df_Dev_str['start'][0][1])/8000, 's')

speaker ID:  FD1 
 transcription:  Go PROCEDURES. 
 start of sample:  1716960 
 end of sample:  1724240 
 number of samples:  7280 
 duration:  0.91 s


In [125]:
# TO-DO: speaker prediction using SAD sampling interval

In [176]:
def intervals(interval_num=0):
    return print('int1_start: ', intervals8k.intervals[interval_num][0], '\n',
                   'int1_end: ', intervals8k.intervals[interval_num][1], '\n',
                    'duration: ', (intervals8k.intervals[interval_num][1]-intervals8k.intervals[interval_num][0])/8000)
intervals(0)

int1_start:  1709760 
 int1_end:  1744880 
 duration:  4.39


In [177]:
audio_int1=[]
for i in range(intervals8k.intervals[0][0],intervals8k.intervals[0][1]):
    audio_int1_slice = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                     start = i, 
                                                    stop = i+8000).astype(np.float32)
    print(i, i+4000)
    audio_int1.append(audio_int1_slice)
#sample shift should be more than 1

1709760 1713760
1709761 1713761
1709762 1713762
1709763 1713763
1709764 1713764
1709765 1713765
1709766 1713766
1709767 1713767
1709768 1713768
1709769 1713769
1709770 1713770
1709771 1713771
1709772 1713772
1709773 1713773
1709774 1713774
1709775 1713775
1709776 1713776
1709777 1713777
1709778 1713778
1709779 1713779
1709780 1713780
1709781 1713781
1709782 1713782
1709783 1713783
1709784 1713784
1709785 1713785
1709786 1713786
1709787 1713787
1709788 1713788
1709789 1713789
1709790 1713790
1709791 1713791
1709792 1713792
1709793 1713793
1709794 1713794
1709795 1713795
1709796 1713796
1709797 1713797
1709798 1713798
1709799 1713799
1709800 1713800
1709801 1713801
1709802 1713802
1709803 1713803
1709804 1713804
1709805 1713805
1709806 1713806
1709807 1713807
1709808 1713808
1709809 1713809
1709810 1713810
1709811 1713811
1709812 1713812
1709813 1713813
1709814 1713814
1709815 1713815
1709816 1713816
1709817 1713817
1709818 1713818
1709819 1713819
1709820 1713820
1709821 1713821
1709822 

1710666 1714666
1710667 1714667
1710668 1714668
1710669 1714669
1710670 1714670
1710671 1714671
1710672 1714672
1710673 1714673
1710674 1714674
1710675 1714675
1710676 1714676
1710677 1714677
1710678 1714678
1710679 1714679
1710680 1714680
1710681 1714681
1710682 1714682
1710683 1714683
1710684 1714684
1710685 1714685
1710686 1714686
1710687 1714687
1710688 1714688
1710689 1714689
1710690 1714690
1710691 1714691
1710692 1714692
1710693 1714693
1710694 1714694
1710695 1714695
1710696 1714696
1710697 1714697
1710698 1714698
1710699 1714699
1710700 1714700
1710701 1714701
1710702 1714702
1710703 1714703
1710704 1714704
1710705 1714705
1710706 1714706
1710707 1714707
1710708 1714708
1710709 1714709
1710710 1714710
1710711 1714711
1710712 1714712
1710713 1714713
1710714 1714714
1710715 1714715
1710716 1714716
1710717 1714717
1710718 1714718
1710719 1714719
1710720 1714720
1710721 1714721
1710722 1714722
1710723 1714723
1710724 1714724
1710725 1714725
1710726 1714726
1710727 1714727
1710728 

1711303 1715303
1711304 1715304
1711305 1715305
1711306 1715306
1711307 1715307
1711308 1715308
1711309 1715309
1711310 1715310
1711311 1715311
1711312 1715312
1711313 1715313
1711314 1715314
1711315 1715315
1711316 1715316
1711317 1715317
1711318 1715318
1711319 1715319
1711320 1715320
1711321 1715321
1711322 1715322
1711323 1715323
1711324 1715324
1711325 1715325
1711326 1715326
1711327 1715327
1711328 1715328
1711329 1715329
1711330 1715330
1711331 1715331
1711332 1715332
1711333 1715333
1711334 1715334
1711335 1715335
1711336 1715336
1711337 1715337
1711338 1715338
1711339 1715339
1711340 1715340
1711341 1715341
1711342 1715342
1711343 1715343
1711344 1715344
1711345 1715345
1711346 1715346
1711347 1715347
1711348 1715348
1711349 1715349
1711350 1715350
1711351 1715351
1711352 1715352
1711353 1715353
1711354 1715354
1711355 1715355
1711356 1715356
1711357 1715357
1711358 1715358
1711359 1715359
1711360 1715360
1711361 1715361
1711362 1715362
1711363 1715363
1711364 1715364
1711365 

1712127 1716127
1712128 1716128
1712129 1716129
1712130 1716130
1712131 1716131
1712132 1716132
1712133 1716133
1712134 1716134
1712135 1716135
1712136 1716136
1712137 1716137
1712138 1716138
1712139 1716139
1712140 1716140
1712141 1716141
1712142 1716142
1712143 1716143
1712144 1716144
1712145 1716145
1712146 1716146
1712147 1716147
1712148 1716148
1712149 1716149
1712150 1716150
1712151 1716151
1712152 1716152
1712153 1716153
1712154 1716154
1712155 1716155
1712156 1716156
1712157 1716157
1712158 1716158
1712159 1716159
1712160 1716160
1712161 1716161
1712162 1716162
1712163 1716163
1712164 1716164
1712165 1716165
1712166 1716166
1712167 1716167
1712168 1716168
1712169 1716169
1712170 1716170
1712171 1716171
1712172 1716172
1712173 1716173
1712174 1716174
1712175 1716175
1712176 1716176
1712177 1716177
1712178 1716178
1712179 1716179
1712180 1716180
1712181 1716181
1712182 1716182
1712183 1716183
1712184 1716184
1712185 1716185
1712186 1716186
1712187 1716187
1712188 1716188
1712189 

1712681 1716681
1712682 1716682
1712683 1716683
1712684 1716684
1712685 1716685
1712686 1716686
1712687 1716687
1712688 1716688
1712689 1716689
1712690 1716690
1712691 1716691
1712692 1716692
1712693 1716693
1712694 1716694
1712695 1716695
1712696 1716696
1712697 1716697
1712698 1716698
1712699 1716699
1712700 1716700
1712701 1716701
1712702 1716702
1712703 1716703
1712704 1716704
1712705 1716705
1712706 1716706
1712707 1716707
1712708 1716708
1712709 1716709
1712710 1716710
1712711 1716711
1712712 1716712
1712713 1716713
1712714 1716714
1712715 1716715
1712716 1716716
1712717 1716717
1712718 1716718
1712719 1716719
1712720 1716720
1712721 1716721
1712722 1716722
1712723 1716723
1712724 1716724
1712725 1716725
1712726 1716726
1712727 1716727
1712728 1716728
1712729 1716729
1712730 1716730
1712731 1716731
1712732 1716732
1712733 1716733
1712734 1716734
1712735 1716735
1712736 1716736
1712737 1716737
1712738 1716738
1712739 1716739
1712740 1716740
1712741 1716741
1712742 1716742
1712743 

...

1741810 1745810
1741811 1745811
1741812 1745812
1741813 1745813
1741814 1745814
1741815 1745815
1741816 1745816
1741817 1745817
1741818 1745818
1741819 1745819
1741820 1745820
1741821 1745821
1741822 1745822
1741823 1745823
1741824 1745824
1741825 1745825
1741826 1745826
1741827 1745827
1741828 1745828
1741829 1745829
1741830 1745830
1741831 1745831
1741832 1745832
1741833 1745833
1741834 1745834
1741835 1745835
1741836 1745836
1741837 1745837
1741838 1745838
1741839 1745839
1741840 1745840
1741841 1745841
1741842 1745842
1741843 1745843
1741844 1745844
1741845 1745845
1741846 1745846
1741847 1745847
1741848 1745848
1741849 1745849
1741850 1745850
1741851 1745851
1741852 1745852
1741853 1745853
1741854 1745854
1741855 1745855
1741856 1745856
1741857 1745857
1741858 1745858
1741859 1745859
1741860 1745860
1741861 1745861
1741862 1745862
1741863 1745863
1741864 1745864
1741865 1745865
1741866 1745866
1741867 1745867
1741868 1745868
1741869 1745869
1741870 1745870
1741871 1745871
1741872 

1742417 1746417
1742418 1746418
1742419 1746419
1742420 1746420
1742421 1746421
1742422 1746422
1742423 1746423
1742424 1746424
1742425 1746425
1742426 1746426
1742427 1746427
1742428 1746428
1742429 1746429
1742430 1746430
1742431 1746431
1742432 1746432
1742433 1746433
1742434 1746434
1742435 1746435
1742436 1746436
1742437 1746437
1742438 1746438
1742439 1746439
1742440 1746440
1742441 1746441
1742442 1746442
1742443 1746443
1742444 1746444
1742445 1746445
1742446 1746446
1742447 1746447
1742448 1746448
1742449 1746449
1742450 1746450
1742451 1746451
1742452 1746452
1742453 1746453
1742454 1746454
1742455 1746455
1742456 1746456
1742457 1746457
1742458 1746458
1742459 1746459
1742460 1746460
1742461 1746461
1742462 1746462
1742463 1746463
1742464 1746464
1742465 1746465
1742466 1746466
1742467 1746467
1742468 1746468
1742469 1746469
1742470 1746470
1742471 1746471
1742472 1746472
1742473 1746473
1742474 1746474
1742475 1746475
1742476 1746476
1742477 1746477
1742478 1746478
1742479 

1743143 1747143
1743144 1747144
1743145 1747145
1743146 1747146
1743147 1747147
1743148 1747148
1743149 1747149
1743150 1747150
1743151 1747151
1743152 1747152
1743153 1747153
1743154 1747154
1743155 1747155
1743156 1747156
1743157 1747157
1743158 1747158
1743159 1747159
1743160 1747160
1743161 1747161
1743162 1747162
1743163 1747163
1743164 1747164
1743165 1747165
1743166 1747166
1743167 1747167
1743168 1747168
1743169 1747169
1743170 1747170
1743171 1747171
1743172 1747172
1743173 1747173
1743174 1747174
1743175 1747175
1743176 1747176
1743177 1747177
1743178 1747178
1743179 1747179
1743180 1747180
1743181 1747181
1743182 1747182
1743183 1747183
1743184 1747184
1743185 1747185
1743186 1747186
1743187 1747187
1743188 1747188
1743189 1747189
1743190 1747190
1743191 1747191
1743192 1747192
1743193 1747193
1743194 1747194
1743195 1747195
1743196 1747196
1743197 1747197
1743198 1747198
1743199 1747199
1743200 1747200
1743201 1747201
1743202 1747202
1743203 1747203
1743204 1747204
1743205 

1743662 1747662
1743663 1747663
1743664 1747664
1743665 1747665
1743666 1747666
1743667 1747667
1743668 1747668
1743669 1747669
1743670 1747670
1743671 1747671
1743672 1747672
1743673 1747673
1743674 1747674
1743675 1747675
1743676 1747676
1743677 1747677
1743678 1747678
1743679 1747679
1743680 1747680
1743681 1747681
1743682 1747682
1743683 1747683
1743684 1747684
1743685 1747685
1743686 1747686
1743687 1747687
1743688 1747688
1743689 1747689
1743690 1747690
1743691 1747691
1743692 1747692
1743693 1747693
1743694 1747694
1743695 1747695
1743696 1747696
1743697 1747697
1743698 1747698
1743699 1747699
1743700 1747700
1743701 1747701
1743702 1747702
1743703 1747703
1743704 1747704
1743705 1747705
1743706 1747706
1743707 1747707
1743708 1747708
1743709 1747709
1743710 1747710
1743711 1747711
1743712 1747712
1743713 1747713
1743714 1747714
1743715 1747715
1743716 1747716
1743717 1747717
1743718 1747718
1743719 1747719
1743720 1747720
1743721 1747721
1743722 1747722
1743723 1747723
1743724 

1744469 1748469
1744470 1748470
1744471 1748471
1744472 1748472
1744473 1748473
1744474 1748474
1744475 1748475
1744476 1748476
1744477 1748477
1744478 1748478
1744479 1748479
1744480 1748480
1744481 1748481
1744482 1748482
1744483 1748483
1744484 1748484
1744485 1748485
1744486 1748486
1744487 1748487
1744488 1748488
1744489 1748489
1744490 1748490
1744491 1748491
1744492 1748492
1744493 1748493
1744494 1748494
1744495 1748495
1744496 1748496
1744497 1748497
1744498 1748498
1744499 1748499
1744500 1748500
1744501 1748501
1744502 1748502
1744503 1748503
1744504 1748504
1744505 1748505
1744506 1748506
1744507 1748507
1744508 1748508
1744509 1748509
1744510 1748510
1744511 1748511
1744512 1748512
1744513 1748513
1744514 1748514
1744515 1748515
1744516 1748516
1744517 1748517
1744518 1748518
1744519 1748519
1744520 1748520
1744521 1748521
1744522 1748522
1744523 1748523
1744524 1748524
1744525 1748525
1744526 1748526
1744527 1748527
1744528 1748528
1744529 1748529
1744530 1748530
1744531 

In [178]:
pb.io.play(audio_int1[20], sample_rate=8000, )

In [129]:
audio_int2[20]

array([ 0.00802612,  0.00723267,  0.0100708 , ..., -0.00543213,
       -0.0022583 ,  0.00759888], dtype=float32)

In [130]:
exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-06-15-22-56'
ckpt_name = 'ckpt_best_loss.pth'
device = 0
model_SID = Model.from_storage_dir(
    exp_dir, consider_mpi=True, checkpoint_name=ckpt_name
)
model_SID.to(device)
model_SID.eval()

ConvAngularPen(
  size=ModelParameterSize(total_count=10694366, trainable_count=10694366, total_bytes=42777464, trainable_bytes=42777464)
  (convlayers): PT_ConvNet(
    size=ModelParameterSize(total_count=5322404, trainable_count=5322404, total_bytes=21289616, trainable_bytes=21289616)
    (layer1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
    (layer2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.01)
      (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): LeakyReLU(negative_slope=0.01)
      (8): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1),

In [131]:
#speaker prediction for 1sample of 1st interval, 1 sample windowing, window length=0.5sec
sid_dict=dict()
fbank_data=[]

f_banks = fbank(audio_int1[20], sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
            stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
            window=scipy.signal.windows.hamming, denoise=False)

f_banks = f_banks**-0.5
print(f_banks)
fbank_data.append(f_banks)
float_fbank = np.float32(fbank_data)
float_fbank = np.squeeze(float_fbank,0)
float_fbank = np.expand_dims(float_fbank, axis=0)
float_fbank = np.expand_dims(float_fbank, axis=0)
float_fbank = torch.from_numpy(float_fbank).to(device)
sid_dict['features'] = (float_fbank)

out = model_SID(sid_dict)

[[3058.23271967  487.03877129  336.40380344 ...  764.97589256
  1134.4617207  2837.67964314]
 [3519.25175837  464.02385084  322.93350351 ... 1019.92434717
  1094.2786784  1533.52530095]
 [2759.30218333  467.86300169  324.02641152 ... 1474.07528217
  1328.05893966 1814.18142314]
 ...
 [2883.46084006  425.51645937  297.60935221 ...  863.60565122
  1237.16701771 2340.54515614]
 [1933.99208432  443.3853499   277.71452233 ...  895.80013577
   829.53182831 1690.80978883]
 [1155.83830797  369.02025899  296.75483491 ...  698.54903948
   602.76304693 3073.45463142]]


In [133]:
torch.argmax(out['prediction'])

tensor(49, device='cuda:0')

In [152]:
#speaker prediction for 1st interval, 1 sample windowing, window length=0.5sec
preds=[]
with torch.no_grad():
    for i in tqdm(range(intervals8k.intervals[0][0],intervals8k.intervals[0][1])):
        audio_int1_slice = pb.io.load_audio(df_Dev_str['audio_path'][0]['observation'], 
                                                         start = i, 
                                                        stop = i+8000).astype(np.float32)
        sid_dict=dict()
        fbank_data=[]

        f_banks = fbank(audio_int1_slice, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                    stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                    window=scipy.signal.windows.hamming, denoise=False)
        f_banks = f_banks**-0.5
        fbank_data.append(f_banks)
        float_fbank = np.float32(fbank_data)
        float_fbank = np.squeeze(float_fbank,0)
        float_fbank = np.expand_dims(float_fbank, axis=0)
        float_fbank = np.expand_dims(float_fbank, axis=0)
        float_fbank = torch.from_numpy(float_fbank).to(device)
        sid_dict['features'] = (float_fbank)

        out = model_SID(sid_dict)
        preds.append(np.argmax(out['prediction'].data.cpu().numpy()))


100%|██████████| 35120/35120 [04:00<00:00, 145.99it/s]


In [154]:
print(preds)

[49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49,

In [211]:
preds[1959]

3

In [192]:
change = np.where(np.diff(preds,prepend=np.nan))[0]
print(change[1:], len(change))

[ 1959  1960  1964  1971  2069  2073  2075  2077  2119  2131  2137  2138
  2155  2156  2225  2227  2228  2239  2241  2242  2273  2293  2294  2300
  2315  2316  2321  2322  2342  2345  2348  2350  2369  2404  2436  2451
  2456  2458  2475  2476  2502  2504  6358  6369  6395  6396  6516  6567
  6569  6585  6591  6595  6601  6602  6603  6748  6750  6757  6764  7177
  7178  7206  7211  7213  7214  7224  7232  7235  7271  7280  7286  7335
  7339  7367  7369  7386  7391  7396  7434  7436  7447  7450  7452  7465
  7470  7493  7500  7502  7504  7506  7509  7521  7524  7526  7535  7544
  7613  7615  7631  7653  7659  7668  7669  7680  7684  7685  7695  7704
  7754  7756  7768  7786  7787  7867  7870  7885  7887  7893  7895 10860
 10861 11018 11027 11145 11148 11157 11161 11164 11167 11170 11174 11175
 11194 11196 11205 11207 11222 11229 11242 11244 11247 11290 11384 11388
 11401 11405 11406 11409 11412 11418 11423 11435 11440 11441 11546 11550
 11562 11564 11567 11569 11575 11578 11583 11590 11

In [184]:
preds=np.array(preds)

In [185]:
c = preds[change]

In [186]:
print(c)

array([49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,
        3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3,
       49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,
        3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3,
       49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,
        3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3,
       49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,
        3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3,
       49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49,
        3, 49,  3, 49,  3, 49,  3, 49,  3, 49,  3, 49, 33,  3, 33,  3, 33,
        3, 33, 49,  3, 49,  3, 49,  3, 49, 33,  3, 49, 33, 49,  3, 49,  3,
       49,  3, 49,  3, 49,  3, 49,  3, 33,  3, 33,  3, 33,  3, 49, 33, 49,
       33,  3, 49,  3, 49,  3, 49,  3, 49,  3, 33, 49,  3, 49, 33, 49, 33,
        3, 49, 33, 49, 33

In [188]:
audio_int1[1959]

array([-0.13140869, -0.12521362, -0.07913208, ..., -0.07815552,
       -0.05917358, -0.0244751 ], dtype=float32)

In [189]:
pb.io.play(audio_int1[1959], sample_rate=8000, )

In [210]:
pb.io.play(audio_int1[1959], sample_rate=8000, )

In [198]:
overlaps = preds[1959:5959]
overlaps

array([ 3, 49, 49, ..., 49, 49, 49])

In [203]:
preds_2 = preds[5959:9959]
for i in preds_2:
    print(i)

49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
4

In [208]:
preds_1 = preds[0:5959]
for i in preds_1:
    print(i)

49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
4

49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
49
4